# Final solution code: 

In [138]:
import numpy as np
import pandas as pd
import networkx as nx
import random

def n_neighbors(graph,node):
    return len([i for i in graph.neighbors(node)])

def check_both_sides(graph,edge):
    return all(n_neighbors(graph,i) > 1 for i in edge)

def risking_my_life_for_these_cubes(rods, verbose = False):
    
    #First count how many repeat edges
    
    repeats = len(rods) - len(set(rods))

    #Initialize Graph and add the edges. At this point repeat edges are cut.
    
    G = nx.Graph()
    G.add_edges_from(rods)
    
    #Iterate through each edge
    
    cut_counter = 0
    edgelist = list(G.edges)
    
    #Randomly shuffle the list of edges so that the rods are considered in a different sequence each time
    random.shuffle(edgelist)
    
    for edge in edgelist:
        
        #First: Are nodes on both ends of edge connected to another node?
        
        if check_both_sides(G,edge):
            
            #Second: After cut, is the graph still connected?
            
            Aftercut = G.copy()
            Aftercut.remove_edge(edge[0],edge[1])
            
            if nx.is_connected(Aftercut):
                
                if verbose:
                    print('Edge {} has been cut!'.format(edge))
                
                #Cut the rod!
                G.remove_edge(edge[0],edge[1])
                cut_counter += 1
                
                if verbose:
                    print('Cut counter: {}'.format(cut_counter))
                
                
                
            else: 
            
                if verbose:
                    print('Cutting this rod ({}) makes the cubes disconnected'.format(edge))
        
        else:
            
            if verbose:
                print ('Cutting this rod ({}) leaves an isolated cube'.format(edge))
            
            continue
            
    return cut_counter + repeats


#Define a function that runs the algorithm a number of iterations for random permutations of the rod sequence

def compute(rods, iterations = 5000, verbose = False):
    solutions = []
    for i in range(iterations):
        solution = risking_my_life_for_these_cubes(rods = rods,verbose = verbose)
        solutions.append(solution)
    return max(solutions)
    

In [144]:
rods = [(1,2),(1,4),(1,5),(1,7),(7,6),(4,6),(2,1),(4,2)]

compute(rods, iterations = 1)

2

# Run the function here:

In [147]:
rods = [(1,2),(1,4),(1,5),(1,7),(7,6),(4,6),(2,1),(4,2)]


compute([(1,4),(1,3),(4,3)])

1

# Algorithm explanation:

The algorithm consists of a few steps:

Step 1:

Check if there are any repeated rods connecting the same 2 cubes.
If such rods exist, remove those rods and save the number of rods removed under "r".

Step 2:
Iterate through each rod. For each rod, check if the cubes at both ends of the rod are connected to at least 2 other cubes. This ensures that if the current rod is cut, the cubes at each end will still be connected to another cube. 

If the condition is not met, the rod cannot be cut. If it is met, then continue to the next step.

Step 3:
Check if after the current rod is cut, whether the entire network of cubes will still remain connected - i.e. every cube must still have at least one path to every other cube in the network. 

If the condition is not met, the rod cannot be cut. If it is met, then the rod can be cut and some platinum is earned!!$$$$

Step 4: After choosing to cut or not cut the rod, the algorithm continues iterating through each rod in the network until every rod has been considered. The function then returns the total number of rods cut (remembering to include the cutting of the duplicate rods in the beginning). 


Sequence of rod consideration:

The algorithm uses a sequential selection of rods to consider (i.e. algorithm considers one rod, then the next, then the next). As the cutting of the rods affects the entire network, the order in which the rods are considered by the algorithm affects whether the optimal solution is obtained. 

Therefore in the compute function, the algorithm is run for 5000 random permutations of the rod sequence, outputting a solution each time. The maximum of all 5000 solutions will be returned as the optimal solution (as we want to cut as many rods as possible). 

However this does not guarantee always finding the optimal solution. Increasing the iterations will make it more likely to find the optimal solution, but with a large number of cubes this method may not be very efficient. 	


# How the algorithm was made:

In [1]:
import numpy as np
import pandas as pd
import networkx as nx #Package for network analysis

Task

● Write a Python program which can tell you how many rods you can cut down. (You only need to provide the
number, no need to enumerate which ones to cut.) Make sure your solution works for all corner cases. The input is
the layout of the rods and cubes. Cubes are identified by integers and each item of the input corresponds to a rod:
it tells which two cubes are connected by that rod.

● Explain the algorithm you used. How does it work? Why is it correct?
Requirements

● The submission has to be written in Python3.

● The solution should implement a function compute(rods) that takes the rods as input and returns the number of
rods that can be removed.

○ The argument rods should be a list of 2-tuples, each tuple corresponding to a rod with the cube identifiers
being the elements of the tuple. For example the input rods = [(1, 2), (1, 3)] means we have three cubes,
identified by 1, 2 and 3, and two rods, one connecting the cubes 1 and 2, and the other connects the cubes 1
and 3.

○ The function compute() should return one value, the number of rods which could be cut. For
example compute([(1, 2), (1, 3), (3, 2)]) should return the value 1.

● Attach a short text file in which you explain your algorithm. It’s a plus if you can provide proof of
correctness.
Example
rods = [(42, 35), (20, 35), (20, 35), (42, 10)]
compute(rods) # Returns 1


In [2]:
rods = [(42, 35), (20, 35), (20, 35), (42, 10)] #input format

# Step 1:

#### Count how many repeated edges (rods) there are, and 'cut' them away.

In [3]:
#The repeat edges will be automatically 'cut' when initializing the graph since repeat edges are not added

repeats = len(rods) - len(set(rods))

# Step 2: 

#### Initialize a graph to represent our cubes and rods.

In [4]:
#Initialize a graph to represent our cubes and rods
#The MultiDiGraph class type is used as we have multiple edges between nodes

G = nx.Graph()

#Add the nodes (cubes) to our graph

G.add_edges_from(rods)

In [5]:
#Check the nodes in the network

G.nodes

NodeView((42, 35, 20, 10))

In [6]:
#Check number of edges

G.number_of_edges()

3

In [7]:
#Check number of nodes

G.number_of_nodes()

4

# Step 3:

#### Iterate through the edges according to the rule:

if: Nodes on both sides of the edges are connected to the other node, proceed.

else: This edge cannot be cut. 

In [8]:
#Get a way to determine number of neighbors a node has

def n_neighbors(node):
    return len([i for i in G.neighbors(node)])

In [9]:
#Get a list of the edges in our graph

list(G.edges())

[(42, 35), (42, 10), (35, 20)]

In [10]:
#Testing out the function

n_neighbors(42)

2

In [12]:
#Define a function to check if both nodes of an edge are connected to another node

def check_both_sides(edge):
    return all(n_neighbors(i) > 1 for i in edge)

# Step 4:

#### Check if after the rod is cut, whether all cubes are still connected by at least one path

In [14]:
#is_connected function checks if graph is connected

TestG = G.copy()

nx.is_connected(TestG)

True

In [15]:
# After removing edge, returns False instead of True

TestG.remove_edge(20,35)

nx.is_connected(TestG)

False

# Step 5:

Iterate until no more rods can be cut. Then add up all the cut rods.

Below we combine everything into one function that takes in one argument (a list containing the rods) and outputs one integer (how many rods can be cut):

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import random

def n_neighbors(graph,node):
    return len([i for i in graph.neighbors(node)])

def check_both_sides(graph,edge):
    return all(n_neighbors(graph,i) > 1 for i in edge)

def compute(rods, verbose = False):
    
    #First count how many repeat edges
    
    repeats = len(rods) - len(set(rods))

    #Initialize Graph and add the edges. At this point repeat edges are cut.
    
    G = nx.Graph()
    G.add_edges_from(rods)
    
    #Iterate through each edge
    
    cut_counter = 0
    edgelist = list(G.edges)
    
    #Randomly shuffle the list of edges so that the rods are considered in a different sequence each time
    random.shuffle(edgelist)
    
    for edge in edgelist:
        
        #First: Are nodes on both ends of edge connected to another node?
        
        if check_both_sides(G,edge):
            
            #Second: After cut, is the graph still connected?
            
            Aftercut = G.copy()
            Aftercut.remove_edge(edge[0],edge[1])
            
            if nx.is_connected(Aftercut):
                
                if verbose:
                    print('Edge {} has been cut!'.format(edge))
                
                #Cut the rod!
                G.remove_edge(edge[0],edge[1])
                cut_counter += 1
                
                if verbose:
                    print('Cut counter: {}'.format(cut_counter))
                
                
                
            else: 
            
                if verbose:
                    print('Cutting this rod ({}) makes the cubes disconnected'.format(edge))
        
        else:
            
            if verbose:
                print ('Cutting this rod ({}) leaves an isolated cube'.format(edge))
            
            continue
            
    return cut_counter + repeats


# In a python script:

In [130]:
import numpy as np
import pandas as pd
import networkx as nx
import random

def n_neighbors(graph,node):
    return len([i for i in graph.neighbors(node)])

def check_both_sides(graph,edge):
    return all(n_neighbors(graph,i) > 1 for i in edge)

def compute(rods, verbose = False):
    
    #First count how many repeat edges
    
    repeats = len(rods) - len(set(rods))

    #Initialize Graph and add the edges. At this point repeat edges are cut.
    
    G = nx.Graph()
    G.add_edges_from(rods)
    
    #Iterate through each edge
    
    cut_counter = 0
    edgelist = list(G.edges)
    
    #Randomly shuffle the list of edges so that the rods are considered in a different sequence each time
    random.shuffle(edgelist)
    
    for edge in edgelist:
        
        #First: Are nodes on both ends of edge connected to another node?
        
        if check_both_sides(G,edge):
            
            #Second: After cut, is the graph still connected?
            
            Aftercut = G.copy()
            Aftercut.remove_edge(edge[0],edge[1])
            
            if nx.is_connected(Aftercut):
                
                if verbose:
                    print('Edge {} has been cut!'.format(edge))
                
                #Cut the rod!
                G.remove_edge(edge[0],edge[1])
                cut_counter += 1
                
                if verbose:
                    print('Cut counter: {}'.format(cut_counter))
                
                
                
            else: 
            
                if verbose:
                    print('Cutting this rod ({}) makes the cubes disconnected'.format(edge))
        
        else:
            
            if verbose:
                print ('Cutting this rod ({}) leaves an isolated cube'.format(edge))
            
            continue
            
    return cut_counter + repeats



#For script

import argparse

parser = argparse.ArgumentParser(description = 'Calculate how many rods can be removed')

parser.add_argument('rods', help = 'List of rods (tuples) connecting the cubes')

args = parser.parse_args()


if __name__ == '__main__':
    
    print(compute(args.rods))

usage: ipykernel_launcher.py [-h] rods
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

In [105]:
testing = [(1,3),(1,3),(1,2),(2,5),(5,2),(3,4),(2,3)]

compute(testing, verbose = True)

Edge (1, 3) has been cut!
Cut counter: 1
Cutting this rod ((1, 2)) leaves an isolated cube
Cutting this rod ((3, 4)) leaves an isolated cube
Cutting this rod ((3, 2)) makes the cubes disconnected
Cutting this rod ((2, 5)) leaves an isolated cube


2